In [1]:
import duckdb
import json

### Repair JSON input

In [2]:
# valid_entries = []
# with open("nextcloud.log.json", "r", encoding="utf-8") as f:
#     for line in f:
#         try:
#             # Process each line as a JSON object
#             valid_entries.append(json.loads(line))  
#         except json.JSONDecodeError:
#             print("Skipping malformed line")

# # Save the repaired JSON
# with open("fixed_nextcloud.log.json", "w", encoding="utf-8") as f:
#     json.dump(valid_entries, f, separators=(',', ':'))

### Ingest data

In [ ]:
%%time

json_input = duckdb.read_json("fixed_nextcloud.log.json")

domain_errors = duckdb.sql("""
                       SELECT *
                       FROM json_input
                       WHERE message LIKE '%Trusted domain error%'
                       -- LIMIT 10
                           """)

gby_user_agent = duckdb.sql("""
                        SELECT remoteAddr, COUNT(*) AS count
                        FROM domain_errors
                        WHERE userAgent LIKE '%Macintosh%'
                        GROUP BY remoteAddr
                        ORDER BY count DESC
                             """).pl()

print(gby_user_agent)